In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import create_model
import inspect, json
from inspect import Parameter

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_API_BASE'] = os.getenv("OPENAI_API_BASE")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE= os.environ.get("OPENAI_API_BASE")

## **Custom agent**

**Define the functions**

In [4]:
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [5]:
abc(2, 3)

-2

In [6]:
def jsonschema(f):
    """
    Generate a JSON schema for the input parameters of the given function.

    Parameters:
        f (FunctionType): The function for which to generate the JSON schema.

    Returns:
        Dict: A dictionary containing the function name, description, and parameters schema.
    """
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
            for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [7]:
def jsonschema1(f):
    """
    Generate a JSON schema for the input parameters of the given function.

    Parameters:
        f (FunctionType): The function for which to generate the JSON schema.

    Returns:
        Dict: A dictionary containing the function name, description, and parameters schema.
    """
    print(inspect.signature(f))
    print(inspect.signature(f).parameters)
    
jsonschema1(abc)

(num1: int, num2: int) -> int
OrderedDict([('num1', <Parameter "num1: int">), ('num2', <Parameter "num2: int">)])


In [ ]:
def foo(a: int, b: int = 10, *args, **kwargs) -> bool:
    pass

jsonschema1(foo)

In [ ]:
abc_json = jsonschema(abc)
abc_json

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    openai_api_base=OPENAI_API_BASE,
    openai_api_key=OPENAI_API_KEY,
    # tream=True,
    temperature=0)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello, how are you?"}
]

# Call the model with the messages
response = chat.invoke(messages)

# Print the response
print(response)
print('\n')
print(response.content)


**Ask GPT**

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    openai_api_base=OPENAI_API_BASE,
    openai_api_key=OPENAI_API_KEY,
    # tream=True,
    temperature=0)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "use abc function to calculate value between 2 and 3"},
]

# Call the model with the messages
response = chat.invoke(messages)

# Print the response
print(response)
print('\n')
print(response.content)


In [ ]:
client = OpenAI()

response = client.chat.completions.create(
  model= model_name,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "compute abc between 2 and 3"},
  ],
)

In [ ]:
response.choices[0].message.content

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    openai_api_base=OPENAI_API_BASE,
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_kwargs={
        "functions": [abc_json],
        "function_call": "auto"
    }
)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "use abc function to calculate value between 2 and 3"},
]

# Call the model with the messages
response = chat.invoke(messages)

# Print the response
print(response)
print('\n')
print(response.content)


In [ ]:
import json

print(response.additional_kwargs)
print(response.additional_kwargs['function_call'])
print(response.additional_kwargs['function_call']['arguments'])
print(response.additional_kwargs['function_call']['name'])
func_name = response.additional_kwargs['function_call']['name']

func_args_str = response.additional_kwargs['function_call']['arguments']
func_args = json.loads(func_args_str)  # 转换为字典
if func_name == 'abc':
    result = abc(**func_args)
print(result)




**Executing the function by extracting the info from the output of the model**

In [ ]:
print(response.choices[0].message.function_call)
print(response.choices[0].message.function_call.arguments)
print(type(response.choices[0].message.function_call.arguments))

In [ ]:
func_name = response.choices[0].message.function_call.name
func_args = json.loads(response.choices[0].message.function_call.arguments)
print("Function name:", func_name)
print("Function arguments:", func_args)
print(type(func_args))

In [ ]:
if func_name == 'abc':
    result = abc(**func_args)
print(result)

## **Using Langchain**

In [ ]:
from langchain_core.tools import tool

@tool
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [ ]:
abc.description

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

import os
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import create_model
import inspect, json
from inspect import Parameter

load_dotenv()

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_API_BASE'] = os.getenv("OPENAI_API_BASE")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE= os.environ.get("OPENAI_API_BASE")


@tool
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # 使用支持函数调用的模型
    openai_api_base=OPENAI_API_BASE,
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)

tools = [abc]

llm_with_tools = llm.bind_tools(tools)

messages = "use abc function to calculate value between 2 and 3"
# [
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "use abc function to calculate value between 2 and 3"},
# ]

# Call the model with the messages
response = llm_with_tools.invoke(messages)
# response = llm.invoke(messages)

# Print the response
print(response)
print('\n')
print(response.content)

In [ ]:
tools = [abc]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
response = llm_with_tools.invoke("Compute abc between 2 and 3")

In [ ]:
response.additional_kwargs

In [ ]:
import getpass
import os

# 如果没有设置 GROQ_API_KEY，提示用户输入 API 密钥
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

# 从 langchain.chat_models 导入 init_chat_model 来初始化 Llama 模型
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

# 使用 @tool 装饰器注册工具函数
@tool
def abc(num1: int, num2: int) -> int:
    "Compute abc between two numbers"
    return 2 * num1 - 2 * num2

# 初始化 Llama 模型，注意 model_provider 设置为 "groq"，并传入温度参数
llm = init_chat_model("llama3-8b-8192", model_provider="groq", temperature=0)

# 将工具绑定到 Llama 模型上
tools = [abc]
llm_with_tools = llm.bind_tools(tools)

# 构造结构化消息，包含 system 和 user 消息，帮助模型理解上下文
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "use abc function to calculate value between 2 and 3"}
]

# 调用模型，模型会根据消息判断是否调用工具
response = llm_with_tools.invoke(messages)

# 输出完整响应、工具调用列表以及生成的文本内容
print("Full Response:", response)
print("\nTool Calls:", response.tool_calls)
print("\nContent:", response.content)


In [1]:
import getpass
import os
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage,SystemMessage


# 如果没有设置 GROQ_API_KEY，则提示用户输入
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

# 使用 @tool 装饰器注册工具函数
@tool
def abcefikfs(a: int, b: int) -> str:
    """abcefikfs do nothing."""
    return 'ffffff'

# 初始化 Llama 模型，使用 Groq 后端
llm = init_chat_model("llama3-8b-8192", model_provider="groq", temperature=0)

# 将工具绑定到模型上
tools = [abcefikfs]
llm_with_tools = llm.bind_tools(tools)

messages=[SystemMessage(content="请只返回最终结果，不要解释，如果没有找到结果，请返回无法找到答案")]
query = "Use function abcefikfs with a=2 and b=3 and return the result."
# print(llm_with_tools.invoke(query).tool_calls)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"abcefikfs": abcefikfs}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
    
messages.append(HumanMessage(content=f"The function returned: {tool_output}. Please summarize."))   
# 第二次调用：将工具输出和额外提示传回给模型，生成最终答案
response = llm.invoke(messages)
print("\nFinal Response:", response.content)


Final Response: The function `abcefikfs` was called with arguments `a=2` and `b=3`. The result of the function is `ffffffffff`.


In [2]:
# 不使用工具，将问题传给模型不用工具处理
messages = [
    SystemMessage(content="请只返回最终结果，不要解释，如果没有找到结果，请返回无法找到答案"),
    HumanMessage(content="Use function abcefikfs with a=2 and b=3 and return the result.")
]

response = llm.invoke(messages)
response.content

'无法找到答案'